<a href="https://colab.research.google.com/github/RohanHanda/Sepsis-Detection-Using-LSTM/blob/main/SepsisDetectorLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Imports**

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Layer, Bidirectional, LSTM, Dense, Dropout, LayerNormalization, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
import numpy as np
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split
import glob
import pandas as pd
import os

**Getting Data, Parsing and Data Preprocessing**

In [ ]:
wget -r -np -nH --cut-dirs=3 \
     -A "*.psv" \
     https://physionet.org/files/challenge-2019/1.0.0/training/


In [ ]:
DATA_DIR = ""

psv_files = glob.glob(os.path.join(DATA_DIR, "*.psv"))
print("Number of patient files:", len(psv_files))


In [ ]:
FEATURES = [
    'HR',        # Heart Rate
    'O2Sat',     # Oxygen Saturation
    'Temp',      # Temperature
    'SBP',       # Systolic BP
    'DBP',       # Diastolic BP
    'Resp',      # Respiratory Rate
    'MAP'        # Mean Arterial Pressure
]

TARGET = 'SepsisLabel'

In [ ]:
def parse_patient_file(filepath):
    df = pd.read_csv(filepath, sep='|')

    # Keep only needed columns (some files may miss some)
    cols = [c for c in FEATURES if c in df.columns]
    cols.append(TARGET)

    df = df[cols]

    # Add patient id
    patient_id = os.path.basename(filepath).replace(".psv", "")
    df['patient_id'] = patient_id

    # Add hour index
    df['hour'] = np.arange(len(df))

    return df

In [ ]:
all_patients = []

for file in psv_files:
    try:
        patient_df = parse_patient_file(file)
        all_patients.append(patient_df)
    except Exception as e:
        print("Error parsing:", file, e)

full_df = pd.concat(all_patients, ignore_index=True)

print("Final shape:", full_df.shape)
full_df.head()

In [ ]:
full_df = full_df.sort_values(['patient_id', 'hour'])

# Forward fill within each patient
full_df[FEATURES] = (
    full_df
    .groupby('patient_id')[FEATURES]
    .ffill()
    .bfill()
)

# Final fallback (rare)
for col in FEATURES:
    full_df[col].fillna(full_df[col].median(), inplace=True)

print(full_df.isna().sum())


In [ ]:
MIN_HOURS = 24

valid_patients = (
    full_df.groupby('patient_id')
    .filter(lambda x: len(x) >= MIN_HOURS)
)

print("After filtering:", valid_patients.shape)


In [ ]:
valid_patients.to_csv("sepsis_timeseries_dataset.csv", index=False)

In [ ]:
df = df.dropna()

scaler = MinMaxScaler()
df[features] = scaler.fit_transform(df[features])

In [1]:
def create_patient_sequences(df, window=24):
    X, y = [], []

    for pid in df['patient_id'].unique():
        patient_data = df[df['patient_id'] == pid]

        values = patient_data[features].values
        labels = patient_data[target].values

        for i in range(window, len(values)):
            X.append(values[i-window:i])
            y.append(labels[i])

    return np.array(X), np.array(y)


In [ ]:
WINDOW = 24

X, y = create_patient_sequences(df, WINDOW)

print(X.shape, y.shape)

In [ ]:
np.save("X.npy", X)
np.save("y.npy", y)

**Attention Layer**

In [ ]:
class ImprovedAttention(Layer):
    """
    Multi-head attention with better gradient flow
    """
    def __init__(self, units=32, num_heads=4):
        super().__init__()
        self.units = units
        self.num_heads = num_heads

    def build(self, input_shape):
        self.W_q = self.add_weight(
            name="query_weight",
            shape=(input_shape[-1], self.units),
            initializer="glorot_uniform"
        )
        self.W_k = self.add_weight(
            name="key_weight",
            shape=(input_shape[-1], self.units),
            initializer="glorot_uniform"
        )
        self.W_v = self.add_weight(
            name="value_weight",
            shape=(input_shape[-1], self.units),
            initializer="glorot_uniform"
        )

    def call(self, x):
        # Multi-head attention
        Q = tf.keras.backend.dot(x, self.W_q)
        K = tf.keras.backend.dot(x, self.W_k)
        V = tf.keras.backend.dot(x, self.W_v)

        # Scaled dot-product attention
        scores = tf.keras.backend.batch_dot(Q, K, axes=[2, 2])
        scores = scores / tf.math.sqrt(tf.cast(self.units, tf.float32))
        attention_weights = tf.keras.backend.softmax(scores, axis=-1)

        # Apply attention to values
        context = tf.keras.backend.batch_dot(attention_weights, V)

        # Global average pooling
        output = tf.reduce_mean(context, axis=1)
        return output

**Custom Metrics**

In [ ]:
def focal_loss(alpha=0.25, gamma=2.0):
    """
    Focal loss focuses training on hard examples
    Better than standard cross-entropy for imbalanced data
    """
    def loss(y_true, y_pred):
        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())

        # Compute focal loss
        pt = y_true * y_pred + (1 - y_true) * (1 - y_pred)
        focal_weight = K.pow(1 - pt, gamma)

        # Cross entropy
        ce = -y_true * K.log(y_pred) - (1 - y_true) * K.log(1 - y_pred)

        # Apply focal weight and alpha
        loss_val = alpha * focal_weight * ce

        return K.mean(loss_val)

    return loss

In [ ]:
class F1Score(tf.keras.metrics.Metric):
    def __init__(self, name='f1_score', **kwargs):
        super().__init__(name=name, **kwargs)
        self.precision_metric = tf.keras.metrics.Precision()
        self.recall_metric = tf.keras.metrics.Recall()

    def update_state(self, y_true, y_pred, sample_weight=None):
        self.precision_metric.update_state(y_true, y_pred, sample_weight)
        self.recall_metric.update_state(y_true, y_pred, sample_weight)

    def result(self):
        p = self.precision_metric.result()
        r = self.recall_metric.result()
        return 2 * ((p * r) / (p + r + K.epsilon()))

    def reset_state(self):
        self.precision_metric.reset_state()
        self.recall_metric.reset_state()

**Model Building**

In [ ]:
def build_improved_model(timesteps, features, lstm_units=[128, 64],
                         attention_units=64, dropout_rate=0.3):
    """
    Improved architecture with:
    - Bidirectional LSTM for better temporal understanding
    - Layer normalization for stable training
    - Skip connections
    - Better regularization
    """
    i = tf.keras.Input(shape=(timesteps, features))

    # First Bidirectional LSTM with layer norm
    x = Bidirectional(LSTM(lstm_units[0], return_sequences=True))(i)
    x = LayerNormalization()(x)
    x = Dropout(dropout_rate)(x)

    # Second Bidirectional LSTM
    x = Bidirectional(LSTM(lstm_units[1], return_sequences=True))(x)
    x = LayerNormalization()(x)
    x = Dropout(dropout_rate)(x)

    # Attention mechanism
    x = ImprovedAttention(units=attention_units)(x)

    # Dense layers
    x = Dense(64, activation="relu")(x)
    x = LayerNormalization()(x)
    x = Dropout(dropout_rate)(x)

    x = Dense(32, activation="relu")(x)
    x = LayerNormalization()(x)

    # Output layer
    x = Dense(1, activation="sigmoid")(x)

    model = Model(i, x)
    return model

**Train Test Split and class imbalance handling**

In [ ]:
def prepare_data_properly(X, y, test_size=0.2, val_size=0.1, random_state=42):
    """
    Proper data splitting to avoid data leakage
    Assumes X is (samples, timesteps, features)
    """
    # First split: train+val vs test
    X_temp, X_test, y_temp, y_test = train_test_split(
        X, y, test_size=test_size, random_state=random_state, stratify=y
    )

    # Second split: train vs val
    val_size_adjusted = val_size / (1 - test_size)
    X_train, X_val, y_train, y_val = train_test_split(
        X_temp, y_temp, test_size=val_size_adjusted,
        random_state=random_state, stratify=y_temp
    )

    return X_train, X_val, X_test, y_train, y_val, y_test

In [ ]:
def oversample_minority_class(X_train, y_train, target_ratio=0.3):
    """
    Simple oversampling by duplicating minority samples with noise
    target_ratio: desired ratio of positive samples
    """
    pos_idx = np.where(y_train == 1)[0]
    neg_idx = np.where(y_train == 0)[0]

    current_ratio = len(pos_idx) / len(y_train)

    if current_ratio >= target_ratio:
        return X_train, y_train

    # Calculate how many positive samples we need
    n_neg = len(neg_idx)
    n_pos_needed = int(n_neg * target_ratio / (1 - target_ratio))
    n_to_add = n_pos_needed - len(pos_idx)

    # Oversample with slight noise
    if n_to_add > 0:
        sampled_idx = np.random.choice(pos_idx, size=n_to_add, replace=True)
        X_oversampled = X_train[sampled_idx]

        # Add small gaussian noise to avoid exact duplicates
        noise = np.random.normal(0, 0.01, X_oversampled.shape)
        X_oversampled = X_oversampled + noise

        X_train = np.vstack([X_train, X_oversampled])
        y_train = np.concatenate([y_train, np.ones(n_to_add)])

        # Shuffle
        shuffle_idx = np.random.permutation(len(y_train))
        X_train = X_train[shuffle_idx]
        y_train = y_train[shuffle_idx]

    return X_train, y_train

**Training**

In [ ]:
def train_improved_model(X, y):
    """
    Complete training pipeline
    """
    # Prepare data properly
    X_train, X_val, X_test, y_train, y_val, y_test = prepare_data_properly(X, y)

    print(f"Train shape: {X_train.shape}, Positive ratio: {y_train.mean():.4f}")
    print(f"Val shape: {X_val.shape}, Positive ratio: {y_val.mean():.4f}")
    print(f"Test shape: {X_test.shape}, Positive ratio: {y_test.mean():.4f}")

    # Apply oversampling to training set only
    X_train, y_train = oversample_minority_class(X_train, y_train, target_ratio=0.2)
    print(f"After oversampling - Train: {X_train.shape}, Positive ratio: {y_train.mean():.4f}")

    # Compute class weights for remaining imbalance
    classes = np.array([0, 1])
    class_weights_array = compute_class_weight(
        class_weight="balanced",
        classes=classes,
        y=y_train
    )
    class_weight_dict = {0: class_weights_array[0], 1: class_weights_array[1]}
    print(f"Class weights: {class_weight_dict}")

    # Build model
    timesteps = X_train.shape[1]
    features = X_train.shape[2]

    model = build_improved_model(
        timesteps=timesteps,
        features=features,
        lstm_units=[128, 64],
        attention_units=64,
        dropout_rate=0.4
    )

    # Compile with improved loss and metrics
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
        loss=focal_loss(alpha=0.75, gamma=2.0),
        metrics=[
            'accuracy',
            tf.keras.metrics.Precision(name='precision'),
            tf.keras.metrics.Recall(name='recall'),
            tf.keras.metrics.AUC(name='auc'),
            F1Score()
        ]
    )

    model.summary()

    # Callbacks
    callbacks = [
        tf.keras.callbacks.EarlyStopping(
            monitor='val_auc',
            patience=10,
            restore_best_weights=True,
            mode='max'
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_auc',
            factor=0.5,
            patience=5,
            min_lr=1e-6,
            mode='max'
        ),
        tf.keras.callbacks.ModelCheckpoint(
            'best_sepsis_model.keras',
            monitor='val_auc',
            save_best_only=True,
            mode='max'
        )
    ]

    # Train
    history = model.fit(
        X_train, y_train,
        validation_data=(X_val, y_val),
        epochs=50,
        batch_size=128,
        class_weight=class_weight_dict,
        callbacks=callbacks,
        verbose=1
    )

    return model, history, X_test, y_test

**Evaluation**

In [ ]:
def evaluate_model(model, X_test, y_test, threshold=0.5):
    """
    Comprehensive evaluation
    """
    from sklearn.metrics import classification_report, roc_auc_score, average_precision_score
    from sklearn.metrics import confusion_matrix

    # Predictions
    y_prob = model.predict(X_test).ravel()
    y_pred = (y_prob >= threshold).astype(int)

    # Metrics
    print("\n" + "="*50)
    print("EVALUATION METRICS")
    print("="*50)
    print(f"\nROC-AUC Score: {roc_auc_score(y_test, y_prob):.4f}")
    print(f"PR-AUC Score: {average_precision_score(y_test, y_prob):.4f}")
    print(f"\nThreshold: {threshold}")
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred, digits=4))

    # Confusion Matrix
    cm = confusion_matrix(y_test, y_pred)
    print("\nConfusion Matrix:")
    print(f"TN: {cm[0,0]}, FP: {cm[0,1]}")
    print(f"FN: {cm[1,0]}, TP: {cm[1,1]}")

    # Calculate additional metrics
    sensitivity = cm[1,1] / (cm[1,1] + cm[1,0]) if (cm[1,1] + cm[1,0]) > 0 else 0
    specificity = cm[0,0] / (cm[0,0] + cm[0,1]) if (cm[0,0] + cm[0,1]) > 0 else 0

    print(f"\nSensitivity (Recall): {sensitivity:.4f}")
    print(f"Specificity: {specificity:.4f}")

    return y_prob, y_pred

**Main**

In [ ]:

# Load your data
X = np.load("X.npy").reshape(644685, 24, 7)
y = np.load("y.npy")

# Train model
model, history, X_test, y_test = train_improved_model(X, y)

# Evaluate
y_prob, y_pred = evaluate_model(model, X_test, y_test, threshold=0.3)

# Find optimal threshold
from sklearn.metrics import precision_recall_curve
precision, recall, thresholds = precision_recall_curve(y_test, y_prob)
f1_scores = 2 * (precision * recall) / (precision + recall + 1e-9)
best_threshold = thresholds[np.argmax(f1_scores)]
print(f"\nOptimal threshold: {best_threshold:.4f}")

# Re-evaluate with optimal threshold
y_prob, y_pred = evaluate_model(model, X_test, y_test, threshold=best_threshold)


Train shape: (451279, 24, 7), Positive ratio: 0.0252
Val shape: (64469, 24, 7), Positive ratio: 0.0252
Test shape: (128937, 24, 7), Positive ratio: 0.0252
After oversampling - Train: (549878, 24, 7), Positive ratio: 0.2000
Class weights: {0: np.float64(0.6249991475393439), 1: np.float64(2.5000136394635146)}


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 24, 7)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 24, 256)        │       139,264 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization             │ (None, 24, 256)        │           512 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 24, 256)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 24, 128)        │       164,352 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_1           │ (None, 24, 128)        │           256 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 24, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ improved_attention              │ (None, 64)             │        24,576 │
│ (ImprovedAttention)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_2           │ (None, 64)             │           128 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ layer_normalization_3           │ (None, 32)             │            64 │
│ (LayerNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 335,425 (1.28 MB)

 Trainable params: 335,425 (1.28 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
4296/4296 ━━━━━━━━━━━━━━━━━━━━ 100s 20ms/step - accuracy: 0.7893 - auc: 0.6700 - f1_score: 0.1284 - loss: 0.0975 - precision: 0.4110 - recall: 0.0784 - val_accuracy: 0.9502 - val_auc: 0.8010 - val_f1_score: 0.1685 - val_loss: 0.0407 - val_precision: 0.1455 - val_recall: 0.2000 - learning_rate: 0.0010
Epoch 2/50
4296/4296 ━━━━━━━━━━━━━━━━━━━━ 86s 20ms/step - accuracy: 0.8255 - auc: 0.8403 - f1_score: 0.4356 - loss: 0.0722 - precision: 0.6080 - recall: 0.3434 - val_accuracy: 0.9309 - val_auc: 0.9276 - val_f1_score: 0.3227 - val_loss: 0.0285 - val_precision: 0.2143 - val_recall: 0.6529 - learning_rate: 0.0010
Epoch 3/50
4296/4296 ━━━━━━━━━━━━━━━━━━━━ 86s 20ms/step - accuracy: 0.9039 - auc: 0.9522 - f1_score: 0.7552 - loss: 0.0431 - precision: 0.7645 - recall: 0.7462 - val_accuracy: 0.9581 - val_auc: 0.9738 - val_f1_score: 0.5022 - val_loss: 0.0191 - val_precision: 0.3583 - val_recall: 0.8394 - learning_rate: 0.0010
Epoch 4/50
4296/4296 ━━━━━━━━━━━━━━━━━━━━ 86s 20ms/step - accur

In [ ]:
model.save("FinalModel.keras")